In [1]:
import sys
sys.path.append('/home/azhir/Dora/pytorch_lightning')
from craftsman.models.autoencoders.michelangelo_autoencoder import MichelangeloAutoencoder

/home/azhir/Dora/pytorch_lightning/craftsman/utils/checkpoint.py:39: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.


In [ ]:
# Define the configuration
config = MichelangeloAutoencoder.Config(
    pretrained_model_name_or_path='/home/azhir/Dora/pytorch_lightning/ckpts/Dora-VAE-1.1/dora_vae_1_1.ckpt',
    use_downsample=True,
    num_latents=256,
    point_feats=3,
    embed_point_feats=False,
    out_dim=1,
    embed_dim=64,
    embed_type='fourier',
    num_freqs=8,
    include_pi=False,
    width=768,
    heads=12,
    num_encoder_layers=8,
    num_decoder_layers=16,
    init_scale=0.25,
    qkv_bias=False,
    use_ln_post=True,
    use_flash=True,
    use_checkpoint=True
)

# Convert Config object to dictionary
config_dict = config.__dict__

# Instantiate the model with the dictionary
model = MichelangeloAutoencoder(cfg=config_dict)
model.eval()  # Set the model to evaluation mode

Loading pretrained shape model from /home/azhir/Dora/pytorch_lightning/ckpts/Dora-VAE-1.1/dora_vae_1_1.ckpt


MichelangeloAutoencoder(
  (embedder): FourierEmbedder()
  (encoder): PerceiverCrossAttentionEncoder(
    (embedder): FourierEmbedder()
    (input_proj): Linear(in_features=54, out_features=768, bias=True)
    (input_proj1): Linear(in_features=54, out_features=768, bias=True)
    (cross_attn): ResidualCrossAttentionBlock(
      (attn): MultiheadCrossAttention(
        (c_q): Linear(in_features=768, out_features=768, bias=False)
        (c_kv): Linear(in_features=768, out_features=1536, bias=False)
        (c_proj): Linear(in_features=768, out_features=768, bias=True)
        (attention): QKVMultiheadCrossAttention()
      )
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (c_fc): Linear(in_features=768, out_features=3072, bias=True)
        (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        (gelu): GELU(approximate='none')
      )
      (ln_3): LayerNorm

In [ ]:
# Generate random latent codes
import torch
latent_code = torch.randn(1, 256, 64)  # [Batch, num_latents, embed_dim]

# Decode the latent codes into shapes
decoded_output = model.decode(latent_code)

# Print the decoded outpu
print('Decoded Output:', decoded_output)

Decoded Output: tensor([[[-0.3500,  0.9348,  0.7684,  ..., -0.4283, -0.0028, -0.3117],
         [-0.3990,  0.8272,  0.3626,  ..., -0.3487, -0.2049,  0.4440],
         [ 0.5666,  0.6042, -0.7087,  ..., -0.2170, -0.1330,  0.1176],
         ...,
         [-0.2403,  0.9295, -0.6673,  ..., -0.5174, -0.3436,  0.4496],
         [ 0.2973, -0.1050,  1.8477,  ..., -0.0144, -0.3656,  1.3114],
         [-0.9137,  0.2099,  2.6271,  ...,  0.0644,  0.2043,  0.8217]]],
       grad_fn=<CheckpointFunctionBackward>)


In [ ]:
decoded_output.shape

git remote set-url origin git@github.com:azhir-physicsx/DORA_VAE.git

git@github.com:azhir-physicsx/DORA_VAE.git

torch.Size([1, 256, 768])